In [3]:
# standard libraries
import os
from datetime import date, datetime
import openpyxl

# imported libraries
import requests
from dotenv import load_dotenv
import pandas as pd
import xlsxwriter

In [4]:
# global vars 

load_dotenv()
output = os.path.join(os.getcwd(), "File Outputs", "LendSaaS")
api_key = os.getenv("LS_APIKEY")
base_url = "https://b2bfinancing.lendtech.io/backend/api/partners"

# specify date range
commencement = "2022-08-03"
run_date = "2022-"
today = str(date.today())


In [5]:
# get leads info
def get_leads(start, end):
    url = f"{base_url}/leads?submittedMinDate={start}&submittedMaxDate={end}"
    leads = requests.get(
        url,
        headers={"x-api-key": api_key}
    )
    return leads.json()

In [6]:
# get offers API
def get_offers(obj, id):
    url = f"{base_url}/offers?leadId={id}"
    offers = requests.get(
        url,
        headers={"x-api-key": api_key}
    )
    obj["offers"] = offers.json()

In [7]:
# funding (funding summary)
def get_funding_snapshots():
    url = f"{base_url}/snapshots/funding"
    payload = {
        "sdate":commencement,
        "edate":today
    }
    
    fundings = requests.post(
        url,
        headers={"x-api-key": api_key},
        json=payload
    )

    return fundings.json()

In [8]:
# account monitoring (asset management style data)
def get_account_snapshots():
    url = f"{base_url}/snapshots/account-monitoring"
    payload = {
        "amStatusId": "1",
        # 1 = Performing
        # 2 = Pending
        "includeClosedDeals": True,
        "includeWriteoffDeals": True
    }
    
    am = requests.post(
        url,
        headers={"x-api-key": api_key},
        json=payload
    )

    return am.json()

# Turtle$$1

In [9]:
# lead level parser for nested info
def parse(obj):

    # search obj
    iso = obj["iso"] or {"isoName": "", "baseFactor": 0, "upfrontComm": 0, "residualComm": 0, "backendComm": 0}

    # iso
    obj["ISO Name"] = iso["isoName"]
    obj["Base Factor"] = iso["baseFactor"]
    obj["Upfront Comm"] = iso["upfrontComm"]
    obj["Residual Comm"] = iso["residualComm"]
    obj["Backend Comm"] = iso["backendComm"]

    return obj


In [10]:
leads = get_leads(commencement, today)

In [11]:
# get lead info
for lead in leads:
    get_offers(lead, lead["leadId"])

In [12]:
servicing = get_account_snapshots()

In [13]:
fundings = get_funding_snapshots()

In [14]:
# parse data and create offers json

offers = []

for lead in leads:
    offers.extend(lead["offers"])
    parse(lead)

In [15]:
selected_offers = list(filter(lambda x: x["selectedTimestamp"] != "0", offers))

In [16]:
ld = pd.DataFrame.from_dict(leads)
of = pd.DataFrame.from_dict(selected_offers)
fn = pd.DataFrame.from_dict(fundings)
sv = pd.DataFrame.from_dict(servicing["deals"])

In [17]:
# convert columns to numbers or dates if possible
dfs = [ld, of, sv, fn] #dd

for df in dfs:
    for col in df.columns:
        if (col.__contains__("Date") | col.__contains__("Time")):
            try:
                df[col] = df[col].apply(datetime.fromtimestamp)
            except TypeError:                
                continue
            finally:
                print(col)
    
    for col in df.columns:
        try:
            df[col] = df[col].apply(pd.to_numeric)
        except:
            continue

fundedDate
entityStartDate
selectedTimestamp
lastPaymentDate
lastReturnDate
minFollowupDate
lastReturnDatePretty
lastPaymentDatePretty
fundedDate
writeoffDate
lastNoteDate
collectionDate
payoffLetterDate
amStatusDate
expectedPayoffDate
defaultDate
fundedDate


In [18]:
fd = fn.merge(of, on="leadId", how="left", suffixes=('', '_remove')).merge(ld, on="leadId", how="left", suffixes=('', '_remove')).merge(sv, on="leadId", how="left", suffixes=('', '_remove'))

In [23]:
summary = {
    "leadId": "Lead ID",
    "entityName": "Company Name",
    "entityCity": "City",
    "entityState": "State",
    "dateSubmitted": "Submitted Date",
    "fundedDate": "Funded Date",
    "amStatus": "AM Status",
    "amStatusDate": "AM Status Date",
    "dealStatus": "Deal Status",
    "category": "Category",
    "advanceAmount": "Advance Amount",
    "rtrAmount": "RTR Amount",
    "buyRate": "Buy Rate",
    "sellRate": "Sell Rate",
    "totalUpfrontFees_remove": "Origination Fee Revenue",
    "totalCommissionAmount": "Commission Expense",
    "paymentAmount": "Payment",
    "paymentFrequency": "Payment Frequency",
    "antDays": "Term (Days)",
    "antTermMonths": "Term (Months)",
    "lastPaymentDatePretty": "Last Payment Date",
    "daysSinceLastPayment": "Days Since Last Payment",
    "variance": "Variance",
    "last7DayVariance": "Variance - Last 7 Days ($)",
    "last7DayVariancePercent": "Variance - Last 7 Days (%)",
    "antCollected30": "Estimated Collections - Last 30 Days", 
    "collected30": "Actual Collections - Last 30 Days", 
    "last30DayVariance": "Variance - Last 30 Days ($)", 
    "last30DayVariancePercent": "Variance - Last 30 Days (%)",
    "countPaymentsAttempted": "Payments Attempted",
    "returnCount": "Bounced Payments",
    "returnRate": "Bounce Rate",
    "totalCollected": "Total Collected",
    "activeFeesBalance": "Active Fees",
}

keep_cols = []

for col in summary.keys():
    keep_cols.append(col)
    
final = fd[keep_cols].rename(columns=summary)

In [25]:
with pd.ExcelWriter(os.path.join(output, f"B2B Servicing Summary {today}.xlsx"), engine="xlsxwriter", date_format="MM/DD/YYYY", datetime_format="MM/DD/YYYY") as writer:
    
    # formats
    wb  = writer.book
    financial = wb.add_format({'num_format': '_(* #,##0.00_);_(* (#,##0.00);_(* "-"??_);_(@_)'})
    percent = wb.add_format({'num_format': '0.0000%'})
        
    # write frames to tabs
    final.to_excel(writer, sheet_name="Summary", index=False)
    # fd.to_excel(writer, sheet_name="Fundings", index=False)
    # sv.to_excel(writer, sheet_name="Servicing", index=False)
    # ld.to_excel(writer, sheet_name="Leads", index=False)
    # dd.to_excel(writer, sheet_name="Deal Desk", index=False)
    
    # FORMAT SHEETS
    ## name sheets
    sum_tab = writer.sheets["Summary"]
    # fd_tab = writer.sheets["Fundings"]
    # sv_tab = writer.sheets["Servicing"]
    # ld_tab = writer.sheets["Leads"]
    # dd_tab = writer.sheets["Deal Desk"]
    
    # worksheets = {fd_tab:fd, sv_tab:sv, ld_tab:ld} # dd_tab:dd, 
    worksheets = {sum_tab:final} 
    
    for sheet in worksheets.keys():
        (max_row, max_col) = worksheets[sheet].shape
        sheet.autofilter(0, 0, max_row, max_col - 1)